In [1]:
import copy
import math
import random
import numpy as np

In [2]:
datas = np.load('data/data.npy')
targets = np.load('data/target.npy')
datas.shape, targets.shape

((10, 20, 20), (10, 20, 20))

In [ ]:
from collections import deque
memory = deque(maxlen=1000)

def append_sample(state, mask, target, reward):
    memory.append((state, mask, target, reward))

In [ ]:
def get_furniture_list(data):
    furnitures = {}
    for i in range(1, 17):
        furnitures[i] = [[]]
        for j in range(20):
            for k in range(20):
                if data[j][k] == i:
                    furnitures[i][j].append(data[j][k])
            furnitures[i].append([])

    for i in range(1, 17):
        for j in range(20, -1, -1):
            if len(furnitures[i][j]) == 0:
                del(furnitures[i][j])
            
    for i in range(16, 0, -1):
        if len(furnitures[i]) == 0:
            del(furnitures[i])
    
    for key in furnitures.keys():
        furnitures[key] = np.array(furnitures[key])
    
    return furnitures

In [ ]:
def get_mask(state, f):
    mask = np.ones((20,20))
    shape = f.shape
    
    zero_list = []
    for i in range(20):
        for j in range(20):
            if state[i][j] != 0:
                mask[i][j] = 0
            else:
                zero_list.append((i,j))
    
    temp_zero_list = copy.deepcopy(zero_list)
    for zero in zero_list:
        z_y, z_x = zero
        if z_y + shape[0] > 20 or z_x + shape[1] > 20:
            mask[z_y][z_x] = 0
            temp_zero_list.remove(zero)
            
    zero_list = copy.deepcopy(temp_zero_list)
    
    for zero in zero_list:
        z_y, z_x = zero
        
        for i in range(z_y, z_y + shape[0]):
            flag = False
            for j in range(z_x, z_x + shape[1]):
                if state[i][j] != 0:
                    mask[z_y][z_x] = 0
                    flag = True
                    break              
            if flag:
                break
            
    return mask

In [ ]:
def get_action_space(mask):
    action_space = []
    for i in range(20):
        for j in range(20):
            if mask[i][j] == 1:
                action_space.append((i,j))
                
    return action_space

In [ ]:
def batch_object(state, furniture, action):
    f = furniture[0][0]
    shape = furniture.shape
    
    for i in range(action[0], action[0] + shape[0]):
        for j in range(action[1], action[1] + shape[1]):
            state[i][j] = f

In [ ]:
def distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

In [ ]:
def calculate_reward(d_pos_list, t_pos_list):
    reward = 0
    
    for d, t in zip(d_pos_list, t_pos_list):
        reward += distance(d, t)
        
    return -reward

In [ ]:
def get_pos_list(state, target, furniture):
    d_pos_list = []
    t_pos_list = []

    for key, f in furniture.items():
        d_pos = (-1,-1)
        t_pos = (-1,-1)

        for i in range(20):
            flag = False
            for j in range(20):
                if state[i][j] == key:
                    d_pos = (i,j)
                    flag = True
                    break
            if flag:
                break

        for i in range(20):
            flag = False
            for j in range(20):
                if target[i][j] == key:
                    t_pos = (i,j)
                    flag = True
                    break
            if flag:
                break

        d_pos_list.append(d_pos)
        t_pos_list.append(t_pos)

    return d_pos_list, t_pos_list